In [7]:
from pandas_profiling import ProfileReport

ModuleNotFoundError: No module named 'pandas_profiling'

In [6]:
seed = 605

import pickle
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from mlxtend.plotting import plot_decision_regions

from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import plot_confusion_matrix
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

def Combine_SkillClusterFamilies(series):
    unique_fam = list(series.unique())
    if 'na' in unique_fam:
        if unique_fam is not ['na']:
            unique_fam.remove('na')
        return ' '.join(unique_fam)
    return ' '.join(unique_fam)

def shorten_4_to_2(self):
    return str(self)[:2]

wdir = '/nfs/turbo/fspp-epi/projects/SkillDemand/'
df_skills = pd.read_csv(wdir + 'data_final/Global_Skill_Families/2015/Global_Skill_Families_2015-05.csv')
df_certs = pd.read_csv(wdir + 'data_raw/Certs/2015/Certs_2015-05.txt', delimiter='\t', encoding='latin1')
df_main = pd.read_csv(wdir + 'data_raw/Main/2015/Main_2015-05.txt', delimiter='\t', encoding='latin1')
df_majors = pd.read_csv(wdir + 'data_final/Skill_Groups/2015/Skill_Groups_2015-05.csv', usecols=['BGTJobId', 'CIP4'])
raw_skills = pd.read_csv(wdir + 'data_raw/Skill/2015/Skills_2015-05.txt', delimiter='\t', encoding='latin1')

df_majors['CIP2'] = df_majors['CIP4'].apply(shorten_4_to_2)

features = ['BGTJobId', 'BestFitMSAName', 'CleanTitle', 'Employer', 'Exp',
       'SectorName', 'SkillClusterFamily', 'SOCName']
features_no_skill = ['BGTJobId', 'BestFitMSAName', 'CleanTitle', 'Employer', 'Exp',
       'SectorName', 'SOCName']

df = df_main[df_main.Edu==16][features_no_skill]
df_sample = df.sample(frac=0.3, random_state=1) # Sample DF or else we're gonna have an error

labelled_df_sample = df_sample.merge(df_majors[['BGTJobId', 'CIP2', 'CIP4']], on='BGTJobId')
labelled_df_sample = labelled_df_sample.replace(False, 0) # Booleans -> Int
labelled_df_w_skills = labelled_df_sample.merge(raw_skills[['BGTJobId','SkillClusterFamily']], on='BGTJobId')
labelled_data = labelled_df_w_skills.groupby(['BGTJobId', 'BestFitMSAName', 'CleanTitle', 'Employer', 'Exp',
       'SectorName', 'SOCName'])['SkillClusterFamily'].apply(Combine_SkillClusterFamilies).reset_index()

In [ ]:
# 70,000 is a lot -- Going to use 5,000 for this workshop

In [ ]:
features_df = labelled_data.sample(5000, random_state = seed) # features df has job id, x df doesn't 
df_labels = features_df.merge(df_majors[['BGTJobId', 'CIP2', 'CIP4']], on='BGTJobId')[['BGTJobId', 'CIP2', 'CIP4']]

In [ ]:
y_df = pd.DataFrame()
labels_CIP = [['is_business', '52'], ['is_compsci', '11'], 
              ['is_comms', '9.'], ['is_publicadmin', '44'],
['is_business_admin_02', 52.02], ['is_computer_science_07', 11.07], 
['is_comm_01', 9.01], ['is_social_work', 44.07]]
for entry in labels_CIP:
    if isinstance(entry[1], float):
        y_df[entry[0]] = df_labels.groupby(['BGTJobId']).apply(lambda x: (any(x.CIP4 == entry[1])))
    else:
        y_df[entry[0]] = df_labels.groupby(['BGTJobId']).apply(lambda x: (any(x.CIP2 == entry[1])))

In [ ]:
y_df

# A Haphazard Journey into Machine Learning and Natural Language Processing
## "Motivational Quote Here"

## Learning Objectives

At the end of this tutorial, you will be able to:
- Describe machine learning's added value to our project
- Outline the basic steps of a machine learning project
- Describe and be able to differentiate between supervised and unsupervised learning
- Describe how features are extracted from text data
- Describe the model validation process

## Missing Majors Problem

In [39]:
overall_df = features_df.merge(y_df, on='BGTJobId')
overall_df.head()

,BGTJobId,BestFitMSAName,CleanTitle,Employer,Exp,SectorName,SOCName,SkillClusterFamily,is_business,is_compsci,is_comms,is_publicadmin,is_business_admin_02,is_computer_science_07,is_comm_01,is_social_work
0,37904664186,"San Francisco-Oakland-Hayward, CA","General Manager, Hockey In",Sportvision,8.0,na,General and Operations Managers,Media and Writing Business Industry Knowledge ...,True,False,False,False,True,False,False,False
1,37902269463,"Provo-Orem, UT","Regional Director, Installation","Vivint, Inc",5.0,Administrative and Support and Waste Managemen...,Sales Managers,Human Resources Education and Training Busines...,True,False,False,False,True,False,False,False
2,37903093614,"Bridgeport-Stamford-Norwalk, CT",Assurance,McGladrey,-999.0,"Professional, Scientific, and Technical Services",Accountants and Auditors,Finance,True,False,False,False,False,False,False,False
3,37895498193,"Cleveland-Elyria, OH",Area Manager,XPO Logistics,4.0,"Professional, Scientific, and Technical Services",General and Operations Managers,Industry Knowledge Media and Writing Supply Ch...,True,False,False,False,True,False,False,False
4,37901172920,"Austin-Round Rock, TX",Desktop Administrator Reopened,Texas Comptroller Of Public Accounts,4.0,na,Computer User Support Specialists,"Information Technology Business Maintenance, R...",False,True,False,False,False,True,False,False


In [45]:
feat_to_show = ['CleanTitle', 'Employer', 'SectorName', 'Exp', 'SOCName', 'SkillClusterFamily']
overall_df[overall_df.is_compsci == True][feat_to_show].head()

,CleanTitle,Employer,SectorName,Exp,SOCName,SkillClusterFamily
4,Desktop Administrator Reopened,Texas Comptroller Of Public Accounts,na,4.0,Computer User Support Specialists,"Information Technology Business Maintenance, R..."
8,Principal Engineer,Navteq Corporation,na,8.0,"Engineers, All Other",Information Technology Engineering
9,Senior Lan Administrator - Code Sgnew,Segal Rogerscasey,"Professional, Scientific, and Technical Services",5.0,Network and Computer Systems Administrators,Information Technology Health Care Engineering...
13,Contact Center Software Development Consultant...,Voya Financial,Finance and Insurance,-999.0,"Software Developers, Applications",Information Technology Marketing and Public Re...
15,Business Systems Analyst Mckesson In,McKesson Corporation,Wholesale Trade,8.0,Computer Systems Analysts,Health Care Business Information Technology Hu...


In [ ]:
overall_df[overall_df.is_compsci == True][feat_to_show].

# Brief Overview of an ML Project


In [48]:
# Taken from _Hands-on Machine Learning with Sci-kit learn_

## Defining the problem

- Defining end goals
- Checking assumptions
- Selecting performance measures

## Exploratory Analysis

- Have a look at data structure
- Visualize data
- Look for correlations

## Preparing the data for ML

- Data cleaning
- Handling text and categorical variables
- Feature scaling (standardize feature units)

## Training and selecting a model
- Training and evaluating on the training set
- Evaluating using cross-validation
- Analyzing best models and their errors

## Fine-tuning the model
- Grid/randomized search
- Ensemble methods

# Baseline Classification Methods

In [ ]:
For each, discuss: 
What is the intuition? / What is the algorithm?
What is it good for?
How does it perform on our data?

In [ ]:
## Logistic Regression



In [ ]:
Support Vector Machine

In [ ]:
K-Nearest Neighbors

In [ ]:
Decision Trees

Quick break

In [ ]:
Review:

# Ensemble Learning
_Cheesy quote here about many dumb learners are better than one smart one_

In [ ]:
Intuiion
What is it good for?
How does it perform?

In [4]:
Bagging

In [5]:
Boosting

SyntaxError: invalid syntax (<ipython-input-5-5e2c66354edb>, line 1)

Model selection
http://ml-ensemble.com/info/tutorials/start.html#model-selection-guide

In [ ]:
Our Data

# Representing text as numbers

# Evaluation Metrics
_How do we know our classifier is good enough?_